# Tool calling

:::tip Compatibility
Tool calling is only available with [supported models](/docs/integrations/chat/).
:::

[Tool calling](/docs/modules/model_io/chat/function_calling) allows a model to respond to a given prompt by generating output that matches a user-defined schema.
By supplying the model with a schema that matches up with a [LangChain tool's](/docs/modules/agents/tools/) signature, along with a name and description of what the tool does, we can get the model
to reliably generate valid input. We can take advantage of this structured output, combined with the fact that you can bind multiple tools to a [tool calling chat model](/docs/integrations/chat/) and
allow the model to choose which one to call, to create an agent that repeatedly calls tools and receives results until a query is resolved.

This is a more generalized version of the [OpenAI tools agent](/docs/modules/agents/agent_types/openai_tools_agent/), which was designed for OpenAI's specific style of
tool calling. It uses LangChain's [ToolCall](https://api.js.langchain.com/types/langchain_core_messages_tool.ToolCall.html) representation to support a wider range of
provider implementations, such as [Anthropic](/docs/integrations/chat/anthropic/), [Google Gemini](/docs/integrations/chat/google_vertex_ai), and [Mistral](/docs/integrations/chat/mistral/)
in addition to [OpenAI](/docs/integrations/chat/openai/).

## Setup

Most models that support tool calling can be used in this agent. See [this list](/docs/integrations/chat/) for the most up-to-date information.

This demo also uses [Tavily](https://app.tavily.com), but you can also swap in another [built in tool](/docs/integrations/platforms).
You'll need to sign up for an API key and set it as `process.env.TAVILY_API_KEY`.

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" additionalDependencies="@langchain/community" />
```

## Initialize Tools

We will first create a tool that can search the web:

In [1]:
import { TavilySearchResults } from "@langchain/community/tools/tavily_search";

// Define the tools the agent will have access to.
const tools = [new TavilySearchResults({ maxResults: 1 })];

## Create Agent

Next, let's initialize our tool calling agent:

In [ ]:
import { AgentExecutor, createToolCallingAgent } from "langchain/agents";
import { ChatOpenAI } from "@langchain/openai";
import { ChatPromptTemplate } from "@langchain/core/prompts";

// Prompt template must have "input" and "agent_scratchpad input variables"
const prompt = ChatPromptTemplate.fromMessages([
  ["system", "You are a helpful assistant"],
  ["placeholder", "{chat_history}"],
  ["human", "{input}"],
  ["placeholder", "{agent_scratchpad}"],
]);

const agent = await createToolCallingAgent({
  llm,
  tools,
  prompt,
});

## Run Agent

Now, let's initialize the executor that will run our agent and invoke it!

In [ ]:
const agentExecutor = new AgentExecutor({
  agent,
  tools,
});

const result = await agentExecutor.invoke({
  input: "what is LangChain?",
});

console.log(result);

:::tip
[LangSmith trace](https://smith.langchain.com/public/5c125a7e-0df5-41ec-96bf-3c13dc3a53f8/r)
:::

## Using with chat history

This type of agent can optionally take chat messages representing previous conversation turns. For more details, see [this section of the agent quickstart](/docs/modules/agents/quick_start#adding-in-memory).

In [ ]:
import { AIMessage, HumanMessage } from "@langchain/core/messages";

const result2 = await agentExecutor.invoke({
  input: "what's my name?",
  chat_history: [
    new HumanMessage("hi! my name is cob"),
    new AIMessage("Hello Cob! How can I assist you today?"),
  ],
});

console.log(result2);